In [1]:
import requests
from bs4 import BeautifulSoup 

import pandas as pd
import numpy as np
import re

In [2]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [3]:
page = "https://www.pro-football-reference.com/years/2021/fantasy.htm"
page_tree = requests.get(page, headers=headers)
page_soup = BeautifulSoup(page_tree.content, 'html.parser')

In [64]:
int(page_soup.find_all("td")[5 + (4 * 32)].text)

16

In [15]:
player_list = []
team_list = []
fantpos_list = []
age_list = []
g_list = []
cmp_list = []
pass_att_list = []
pass_yds_list = []
pass_tds_list = []
int_list = []
rush_att_list = []
rush_yds_list = []
rush_tds_list = []
tgt_list = []
rec_list = []
rec_yards_list = []
rec_td_list = []
ppr_list = []
pos_rank_list = []
ov_rank_list = []

In [4]:
years = np.arange(2013, 2022)
ff_data = pd.DataFrame()

for year in years:
    year_page = f"https://www.pro-football-reference.com/years/{year}/fantasy.htm"
    year_page_tree = requests.get(year_page, headers=headers)
    year_page_soup = BeautifulSoup(year_page_tree.content, 'html.parser')
    
    year_data = year_page_soup.find_all("td")
    
    player_list = []
    team_list = []
    fantpos_list = []
    age_list = []
    g_list = []
    cmp_list = []
    pass_att_list = []
    pass_yds_list = []
    pass_tds_list = []
    int_list = []
    rush_att_list = []
    rush_yds_list = []
    rush_tds_list = []
    tgt_list = []
    rec_list = []
    rec_yards_list = []
    rec_td_list = []
    ppr_list = []
    pos_rank_list = []
    ov_rank_list = []
    
    for i in np.arange(0, (int(len(year_data)) / 32)).tolist():
        i = int(i)
        player_list.append(re.sub('[^A-Za-z0-9]+', '', year_data[0 + i*32].text))
        team_list.append(year_data[1 + i*32].text)
        fantpos_list.append(year_data[2 + i*32].text)
        age_list.append(int(year_data[3 + i*32].text))
        g_list.append(int(year_data[4 + i*32].text))
        cmp_list.append((year_data[6 + i*32].text))
        pass_att_list.append((year_data[7 + i*32].text))
        pass_yds_list.append((year_data[8 + i*32].text))
        pass_tds_list.append((year_data[9 + i*32].text))
        int_list.append((year_data[10 + i*32].text))
        rush_att_list.append((year_data[11 + i*32].text))
        rush_yds_list.append((year_data[12 + i*32].text))
        rush_tds_list.append((year_data[14 + i*32].text))
        tgt_list.append((year_data[15 + i*32].text))
        rec_list.append((year_data[16 + i*32].text))
        rec_yards_list.append((year_data[17 + i*32].text))
        rec_td_list.append((year_data[19 + i*32].text))
        ppr_list.append((year_data[26 + i*32].text))
        pos_rank_list.append((year_data[30 + i*32].text))
        ov_rank_list.append((year_data[31 + i*32].text))
        
    season_data = pd.DataFrame({"Player": player_list, "Team": team_list, "FantPos": fantpos_list,
                                "Age": age_list, "G": g_list, "Cmp": cmp_list, "PassAtt": pass_att_list,
                                "PassYds": pass_yds_list, "PassTD": pass_tds_list, "Int": int_list,
                                "RushAtt": rush_att_list, "RushYds": rush_yds_list, "RushTD": rush_tds_list,
                                "Tgt": tgt_list, "Rec": rec_list, "RecYds": rec_yards_list, "RecTD": rec_td_list,
                                "PPR": ppr_list, "PosRank": pos_rank_list, "OvRank": ov_rank_list, "Year": year})
    ff_data = pd.concat([ff_data, season_data], ignore_index=True)

In [5]:
pd.set_option('display.max_columns', 500)
pg_columns = ff_data.columns.tolist()[5:18]
for column in pg_columns:
    ff_data[column] = np.round(((pd.to_numeric(ff_data[column], errors='coerce')) / ff_data["G"]), 2)

In [6]:
ff_data["Team"].unique()

array(['KAN', 'PHI', 'DEN', 'CHI', 'NOR', 'SEA', 'CLE', 'DET', 'CIN',
       'PIT', 'GNB', 'DAL', 'MIN', 'TEN', 'SFO', 'BUF', 'SDG', 'HOU',
       'WAS', 'TAM', 'ATL', 'CAR', 'ARI', 'IND', 'STL', 'NWE', 'MIA',
       'BAL', 'JAX', 'OAK', 'NYG', 'NYJ', '2TM', '3TM', '4TM', 'LAR',
       'LAC', 'LVR'], dtype=object)

In [7]:
squads = ff_data["Team"].replace({'DAL': "Cowboys", 'PIT': "Steelers", 'SEA': "Seahawks", 
                                  'CHI': "Bears", 'HOU': "Texans", 'GNB': "Packers", 
                                  'DEN': "Broncos", 'NWE': "Patriots", 'KAN': "Chiefs",
                                  'IND': "Colts", 'NYG': "Giants", 'BAL': "Ravens", 
                                  'ATL': "Falcons", 'PHI': "Eagles", 'MIA': "Dolphins", 
                                  'SDG': "Chargers", 'CIN': "Bengals", 'TAM': "Buccaneers",
                                  'NOR': "Saints", 'WAS': "WFT", 'CAR': "Panthers", 'DET': "Lions", 
                                  'MIN': "Vikings", 'SFO': "49ers", 'TEN': "Titans", 'NYJ': "Jets", 
                                  'BUF':"Bills", 'ARI': "Cardinals", 'OAK': "Raiders", 'JAX': "Jaguars", 
                                  'CLE': "Browns", 'STL': "Rams", 'LAR': "Rams", 'LAC': "Chargers", 
                                  'LVR': "Raiders"})

In [8]:
ff_data["Team"] = squads

In [9]:
ff_data

,Player,Team,FantPos,Age,G,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,Tgt,Rec,RecYds,RecTD,PPR,PosRank,OvRank,Year
0,JamaalCharles,Chiefs,RB,27,15,0.00,0.00,0.00,0.00,0.00,17.27,85.80,0.80,6.93,4.67,46.20,0.47,25.20,1,1,2013
1,LeSeanMcCoy,Eagles,RB,25,16,0.00,0.00,0.00,0.00,0.00,19.62,100.44,0.56,4.00,3.25,33.69,0.12,20.66,2,2,2013
2,PeytonManning,Broncos,QB,37,16,28.12,41.19,342.31,3.44,0.62,2.00,-1.94,0.06,0.00,0.00,0.00,0.00,25.62,1,3,2013
3,MattForte,Bears,RB,28,16,0.00,0.00,0.00,0.00,0.00,18.06,83.69,0.56,5.94,4.62,37.12,0.19,21.08,3,4,2013
4,JimmyGraham,Saints,TE,27,16,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.88,5.38,75.94,1.00,18.97,1,5,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5502,TravisBenjamin,49ers,,32,10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.00,0.00,-0.20,260,,2021
5503,TrentonCannon,2TM,RB,27,12,0.00,0.00,0.00,0.00,0.00,0.25,0.33,0.00,0.00,0.00,0.00,0.00,-0.13,179,,2021
5504,JohnWolford,Rams,QB,26,3,0.33,1.33,1.67,0.00,0.33,0.67,-0.33,0.00,0.00,0.00,0.00,0.00,-0.63,84,,2021
5505,JoshRosen,Falcons,QB,24,4,0.50,2.75,4.75,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.80,85,,2021


In [10]:
ff_data.to_csv("ff_points.csv", encoding='utf-8', index=False)